# Utility

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras import backend as K

Using TensorFlow backend.


In [2]:
class utility:

    def read_CSV(self, filename):
        df = pd.read_csv(filename)
        return df

    def get_text_label(self, df):
        texts = []  # list of text samples
        labels = []  # list of label ids
        for index, row in df.iterrows():
            if isinstance(row['sentence'], float):
                texts.append(str(row['sentence']))
            else:
                texts.append(row['sentence'])

            labels.append(row['label'])

        return texts, labels

    def tokenize_texts(self, texts):
        tokenizer = Tokenizer(num_words=10000)
        tokenizer.fit_on_texts(texts)

        return tokenizer

    def padding_texts(self, texts, maxlen):

        texts = pad_sequences(texts, padding='post', maxlen=maxlen)

        return texts

    def get_metric(self, y_true, y_pred):
        accuracyScore = accuracy_score(y_true, y_pred)

        # binary: Only report results for the class specified by pos_label. This is applicable only if targets (y_{true,pred}) are binary.
        precisionScoreBinary = precision_score(y_true, y_pred, average='binary')
        recallScoreBinary = recall_score(y_true, y_pred, average='binary')
        f1ScoreBinary = f1_score(y_true, y_pred, average='binary')

        return accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary

    def print_metric(self, accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary):
        print("Accuracy: " + str(accuracyScore))
        print("Precision: " + str(precisionScoreBinary))
        print("Recall: " + str(recallScoreBinary))
        print("F1-Score: " + str(f1ScoreBinary))
        print(str(accuracyScore) + "," + str(precisionScoreBinary) + "," + str(recallScoreBinary) + "," + str(
            f1ScoreBinary))

    def get_testing_metric(self, y_test, y_pred):
        # metric for Testing Data
        # print("Testing Data")
        accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary = self.get_metric(y_test, y_pred)
        # print()

        return accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary

    def write_df_csv(self, df, out_path):
        df.to_csv(out_path, index=False)

    def create_embedding_matrix(self, filepath, word_index, embedding_dim):
        vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
        embedding_matrix = np.zeros((vocab_size, embedding_dim))

        with open(filepath, encoding="utf8") as f:
            for line in f:
                word, *vector = line.split()
                if word in word_index:
                    idx = word_index[word]
                    embedding_matrix[idx] = np.array(
                        vector, dtype=np.float32)[:embedding_dim]

        return embedding_matrix

    def get_max_length_of_sentences(self, texts):
        maxlength = 0
        for text in texts:
            if (len(text.split()) > maxlength):
                maxlength = len(text.split())

        return maxlength

    def get_training_trial_data(self, textsTraining, labelsTraining, textsTrial, labelsTrial, glovePath):
        textsTraining, textsTesting = np.asarray(textsTraining), np.asarray(textsTrial)
        y_train, y_val = np.asarray(labelsTraining), np.asarray(labelsTrial)

        # Tokenize words
        tokenizer = self.tokenize_texts(textsTraining)
        X_train = tokenizer.texts_to_sequences(textsTraining)
        X_val = tokenizer.texts_to_sequences(textsTesting)

        # Adding 1 because of reserved 0 index
        vocab_size = len(tokenizer.word_index) + 1

        # get maxlen
        maxlen = self.get_max_length_of_sentences(textsTraining)

        # Pad sequences with zeros
        X_train = self.padding_texts(X_train, maxlen)
        X_val = self.padding_texts(X_val, maxlen)

        embedding_matrix = []
        embedding_matrix.append(self.create_embedding_matrix(glovePath[0], tokenizer.word_index, 50))
        embedding_matrix.append(self.create_embedding_matrix(glovePath[1], tokenizer.word_index, 100))
        embedding_matrix.append(self.create_embedding_matrix(glovePath[2], tokenizer.word_index, 200))
        embedding_matrix.append(self.create_embedding_matrix(glovePath[3], tokenizer.word_index, 300))

        return X_train, X_val, y_train, y_val, vocab_size, maxlen, embedding_matrix

    def Average(self, list):
        return sum(list) / len(list)
    
    def recall_m(self, y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision_m(self, y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def f1_m(self, y_true, y_pred):
        precision = self.precision_m(y_true, y_pred)
        recall = self.recall_m(y_true, y_pred)
        return 2*((precision*recall)/(precision+recall+K.epsilon()))

# Finite State Machine

In [3]:
import random

def FSM():
    fsm = {}
    fsm[0] = {'src': 0, 'dst': 1, 'layer': 'embedding_layer', 'next_path': [1]}
    fsm[1] = {'src': 1, 'dst': 2, 'layer': 'convolutional_layer', 'next_path': [2, 3, 5]}
    fsm[2] = {'src': 2, 'dst': 2, 'layer': 'convolutional_layer', 'next_path': [2, 3, 5]}
    fsm[3] = {'src': 2, 'dst': 3, 'layer': 'maxpooling_layer', 'next_path': [4]}
    fsm[4] = {'src': 3, 'dst': 2, 'layer': 'convolutional_layer', 'next_path': [2, 3, 5]}
    fsm[5] = {'src': 2, 'dst': 4, 'layer': 'global_maxpooling_layer', 'next_path': [6, 7]}
    fsm[6] = {'src': 4, 'dst': 5, 'layer': 'dense_layer', 'next_path': [8, 9, 11]}
    fsm[7] = {'src': 4, 'dst': 6, 'layer': 'dropout_layer', 'next_path': [10, 12]}
    fsm[8] = {'src': 5, 'dst': 5, 'layer': 'dense_layer', 'next_path': [8, 9, 11]}
    fsm[9] = {'src': 5, 'dst': 6, 'layer': 'dropout_layer', 'next_path': [10, 12]}
    fsm[10] = {'src': 6, 'dst': 5, 'layer': 'dense_layer', 'next_path': [8, 9, 11]}
    fsm[11] = {'src': 5, 'dst': 7, 'layer': 'output_layer', 'next_path': []}
    fsm[12] = {'src': 6, 'dst': 7, 'layer': 'output_layer', 'next_path': []}

    return fsm

def mutateFSM():
    fsm = {}
    fsm['convolutional_layer'] = {'before': ['convolutional_layer'],
                                  'after': ['convolutional_layer'], 'change': 'maxpooling_layer'}
    fsm['maxpooling_layer'] = {'before': ['convolutional_layer'],
                               'after': ['convolutional_layer'], 'change': 'convolutional_layer'}
    fsm['dense_layer'] = {'before': ['global_maxpooling_layer', 'dense_layer'],
                          'after': ['dense_layer'], 'change': 'dropout_layer'}
    fsm['dropout_layer'] = {'before': ['global_maxpooling_layer', 'dense_layer'],
                            'after': ['dense_layer'], 'change': 'dense_layer'}

    return fsm

def addFSM():
    fsm = {}
    fsm['convolutional_layer'] = {'before': ['convolutional_layer'],
                                  'add': ['convolutional_layer', 'maxpooling_layer']}
    fsm['maxpooling_layer'] = {'before': ['convolutional_layer'],
                               'add': ['convolutional_layer']}
    fsm['dense_layer'] = {'before': ['global_maxpooling_layer', 'dense_layer'],
                          'add': ['dense_layer', 'dropout_layer']}
    fsm['dropout_layer'] = {'before': ['global_maxpooling_layer', 'dense_layer'],
                            'add': ['dense_layer']}

    return fsm

def addConvLayer(idx, toolbox, toolboxes, defaultVal, layerparameters):
    toolbox.register('num_filters' + str(idx), layerparameters['num_filters'][0],
                     layerparameters['num_filters'][1], layerparameters['num_filters'][2])
    toolboxes.append(toolbox.__getattribute__('num_filters' + str(idx)))
    toolbox.register('kernel_size' + str(idx), layerparameters['kernel_size'][0],
                     layerparameters['kernel_size'][1], layerparameters['kernel_size'][2])
    toolboxes.append(toolbox.__getattribute__('kernel_size' + str(idx)))
    toolbox.register('conv_activation_func' + str(idx), layerparameters['conv_activation_func'][0],
                     layerparameters['conv_activation_func'][1])
    toolboxes.append(toolbox.__getattribute__('conv_activation_func' + str(idx)))
    toolbox.register('conv_init_mode' + str(idx), layerparameters['conv_init_mode'][0],
                     layerparameters['conv_init_mode'][1])
    toolboxes.append(toolbox.__getattribute__('conv_init_mode' + str(idx)))
    toolbox.register('conv_weight_constraint' + str(idx), layerparameters['conv_weight_constraint'][0],
                     layerparameters['conv_weight_constraint'][1], layerparameters['conv_weight_constraint'][2])
    toolboxes.append(toolbox.__getattribute__('conv_weight_constraint' + str(idx)))

    defaultVal.update({'num_filters' + str(idx): 64})
    defaultVal.update({'kernel_size' + str(idx): 3})
    defaultVal.update({'conv_activation_func' + str(idx): "relu"})
    defaultVal.update({'conv_init_mode' + str(idx): "glorot_uniform"})
    defaultVal.update({'conv_weight_constraint' + str(idx): 3})


def addDenseLayer(idx, toolbox, toolboxes, defaultVal, layerparameters):
    toolbox.register('neurons' + str(idx), layerparameters['neurons'][0],
                     layerparameters['neurons'][1], layerparameters['neurons'][2])
    toolboxes.append(toolbox.__getattribute__('neurons' + str(idx)))
    toolbox.register('dense_activation_func' + str(idx), layerparameters['dense_activation_func'][0],
                     layerparameters['dense_activation_func'][1])
    toolboxes.append(toolbox.__getattribute__('dense_activation_func' + str(idx)))
    toolbox.register('dense_init_mode' + str(idx), layerparameters['dense_init_mode'][0],
                     layerparameters['dense_init_mode'][1])
    toolboxes.append(toolbox.__getattribute__('dense_init_mode' + str(idx)))
    toolbox.register('dense_weight_constraint' + str(idx), layerparameters['dense_weight_constraint'][0],
                     layerparameters['dense_weight_constraint'][1], layerparameters['dense_weight_constraint'][2])
    toolboxes.append(toolbox.__getattribute__('dense_weight_constraint' + str(idx)))

    defaultVal.update({'neurons' + str(idx): 1})
    defaultVal.update({'dense_activation_func' + str(idx): "relu"})
    defaultVal.update({'dense_init_mode' + str(idx): "glorot_uniform"})
    defaultVal.update({'dense_weight_constraint' + str(idx): 3})


def addMaxPoolingLayer(idx, toolbox, toolboxes, defaultVal, layerparameters):
    toolbox.register('pool_size' + str(idx), layerparameters['pool_size'][0],
                     layerparameters['pool_size'][1], layerparameters['pool_size'][2])
    toolboxes.append(toolbox.__getattribute__('pool_size' + str(idx)))

    defaultVal.update({'pool_size' + str(idx): 5})


def addDropoutLayer(idx, toolbox, toolboxes, defaultVal, layerparameters):
    toolbox.register('dropout_rate' + str(idx), layerparameters['dropout_rate'][0],
                     layerparameters['dropout_rate'][1], layerparameters['dropout_rate'][2])
    toolboxes.append(toolbox.__getattribute__('dropout_rate' + str(idx)))

    defaultVal.update({'dropout_rate' + str(idx): 0.2})

def getLayerSize(layer, conv_idx, dense_idx, dropout_idx, maxpooling_idx):
    if layer == 'convolutional_layer':
        conv_idx += 1
    elif layer == 'dense_layer':
        dense_idx += 1
    elif layer == 'dropout_layer':
        dropout_idx += 1
    elif layer == 'maxpooling_layer':
        maxpooling_idx += 1
    return conv_idx, dense_idx, dropout_idx, maxpooling_idx


def getMaxLayerSize(conv_idx, dense_idx, dropout_idx, maxpooling_idx, max_conv_idx, max_dense_idx, max_dropout_idx,
                    max_maxpooling_idx):
    if conv_idx > max_conv_idx:
        max_conv_idx = conv_idx
    if dense_idx > max_dense_idx:
        max_dense_idx = dense_idx
    if dropout_idx > max_dropout_idx:
        max_dropout_idx = dropout_idx
    if maxpooling_idx > max_maxpooling_idx:
        max_maxpooling_idx = maxpooling_idx

    return max_conv_idx, max_dense_idx, max_dropout_idx, max_maxpooling_idx


def addLayerToolboxes(max_conv_idx, max_dense_idx, max_dropout_idx, max_maxpooling_idx, toolbox, toolboxes, defaultVal,
                      layerparameters):
    idx = 0
    while idx < max_conv_idx:
        idx += 1
        addConvLayer(idx, toolbox, toolboxes, defaultVal, layerparameters)

    idx = 0
    while idx < max_dense_idx:
        idx += 1
        addDenseLayer(idx, toolbox, toolboxes, defaultVal, layerparameters)

    idx = 0
    while idx < max_maxpooling_idx:
        idx += 1
        addMaxPoolingLayer(idx, toolbox, toolboxes, defaultVal, layerparameters)

    idx = 0
    while idx < max_dropout_idx:
        idx += 1
        addDropoutLayer(idx, toolbox, toolboxes, defaultVal, layerparameters)


def generateFSM(n_pop, layerparameters, toolbox, toolboxes, defaultVal):
    fsm = FSM()

    path_ind = {}
    max_conv_idx = 0
    max_dense_idx = 0
    max_dropout_idx = 0
    max_maxpooling_idx = 0

    for ind in range(0, n_pop):
        idx = conv_idx = dense_idx = dropout_idx = maxpooling_idx = 0
        path = [fsm[idx]['layer']]
        while len(fsm[idx]['next_path']) != 0:
            idx = random.choice(fsm[idx]['next_path'])
            layer = fsm[idx]['layer']
            path.append(layer)
            conv_idx, dense_idx, dropout_idx, maxpooling_idx = getLayerSize(layer, conv_idx, dense_idx, dropout_idx,
                                                                            maxpooling_idx)

        max_conv_idx, max_dense_idx, max_dropout_idx, max_maxpooling_idx = getMaxLayerSize(conv_idx, dense_idx,
                                                                                           dropout_idx, maxpooling_idx,
                                                                                           max_conv_idx, max_dense_idx,
                                                                                           max_dropout_idx,
                                                                                           max_maxpooling_idx)

        path_ind[ind] = path

    addLayerToolboxes(max_conv_idx, max_dense_idx, max_dropout_idx, max_maxpooling_idx, toolbox, toolboxes, defaultVal,
                      layerparameters)

    return path_ind, max_conv_idx, max_maxpooling_idx, max_dense_idx, max_dropout_idx


def openFSM(df, layerparameters, toolbox, toolboxes, defaultVal):
    path_ind = {}
    fitnesses = []

    hyperparams = [s for s in list(df.columns) if not 'Unnamed' in s]

    max_conv_idx = sum('num_filters' in s for s in hyperparams)
    max_dense_idx = sum('neurons' in s for s in hyperparams)
    max_dropout_idx = sum('dropout_rate' in s for s in hyperparams)
    max_maxpooling_idx = sum('pool_size' in s for s in hyperparams)

    for index, row in df.iterrows():
        path = [s for s in row if 'layer' in str(s)]
        fitness = [s for s in row if str(s).replace('.', '', 1).isdigit()]
        fitnesses.append(tuple([float(fitness[0])]))
        path_ind[index] = path

    addLayerToolboxes(max_conv_idx, max_dense_idx, max_dropout_idx, max_maxpooling_idx, toolbox, toolboxes, defaultVal,
                      layerparameters)

    return path_ind, fitnesses, max_conv_idx, max_maxpooling_idx, max_dense_idx, max_dropout_idx

# CNN

In [4]:
import tensorflow as tf


class CNN:

    def cnn_model(self, vocab_size, maxlen, embedding_matrix, indiv, path):
        model = tf.keras.models.Sequential()
        conv_idx = dense_idx = dropout_idx = maxpooling_idx = 0
        for layer in path:
            if layer == 'embedding_layer':
                model.add(
                    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=indiv['output_dim'],
                                     weights=[embedding_matrix], input_length=maxlen, trainable=True))
            elif layer == 'convolutional_layer':
                conv_idx += 1
                model.add(tf.keras.layers.Conv1D(indiv['num_filters' + str(conv_idx)], indiv['kernel_size' + str(conv_idx)],
                                        kernel_initializer=indiv['conv_init_mode' + str(conv_idx)],
                                        activation=indiv['conv_activation_func' + str(conv_idx)],
                                        kernel_constraint=tf.keras.constraints.max_norm(indiv['conv_weight_constraint' + str(conv_idx)]),
                                        data_format='channels_first'))
            elif layer == 'dense_layer':
                dense_idx += 1
                model.add(tf.keras.layers.Dense(indiv['neurons' + str(dense_idx)],
                                       kernel_initializer=indiv['dense_init_mode' + str(dense_idx)],
                                       activation=indiv['dense_activation_func' + str(dense_idx)],
                                       kernel_constraint=tf.keras.constraints.max_norm(indiv['dense_weight_constraint' + str(dense_idx)])))
            elif layer == 'dropout_layer':
                dropout_idx += 1
                model.add(tf.keras.layers.Dropout(indiv['dropout_rate' + str(dropout_idx)]))
            elif layer == 'maxpooling_layer':
                maxpooling_idx += 1
                model.add(tf.keras.layers.MaxPooling1D(indiv['pool_size' + str(maxpooling_idx)]))
            elif layer == 'global_maxpooling_layer':
                model.add(tf.keras.layers.GlobalMaxPooling1D())
            elif layer == 'output_layer':
                model.add(tf.keras.layers.Dense(1, kernel_initializer=indiv['output_init_mode'], activation='sigmoid'))

        if indiv['optimizer'] == 'sgd':
            opt = tf.keras.optimizers.SGD(lr=indiv['learning_rate'], momentum=indiv['momentum'], decay=0.0,
                                 nesterov=False)
        elif indiv['optimizer'] == 'rmsprop':
            opt = tf.keras.optimizers.RMSprop(lr=indiv['learning_rate'], rho=0.9, epsilon=None, decay=0.0)
        elif indiv['optimizer'] == 'adagrad':
            opt = tf.keras.optimizers.Adagrad(lr=indiv['learning_rate'], epsilon=None, decay=0.0)
        elif indiv['optimizer'] == 'adadelta':
            opt = tf.keras.optimizers.Adadelta(lr=indiv['learning_rate'], rho=0.95, epsilon=None, decay=0.0)
        elif indiv['optimizer'] == 'adam':
            opt = tf.keras.optimizers.Adam(lr=indiv['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=None,
                                  decay=0.0, amsgrad=False)
        elif indiv['optimizer'] == 'adamax':
            opt = tf.keras.optimizers.Adamax(lr=indiv['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=None,
                                    decay=0.0)
        elif indiv['optimizer'] == 'nadam':
            opt = tf.keras.optimizers.Nadam(lr=indiv['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=None,
                                   schedule_decay=0.004)

        util = utility()
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[util.f1_m])

        return model

# Fitness Calculation

In [5]:
import collections
import os
from time import sleep

util = utility()
cnn = CNN()

def FitnessCalculation(individual, cfold, defaultVal, resultsPath, testing_name):
    indiv = collections.OrderedDict()
    i = 0
    for key in defaultVal.keys():
        indiv[key] = individual[i]
        i += 1

    path = individual[len(defaultVal):len(individual)]

    return crossfold(indiv, path, cfold, resultsPath, testing_name)


def crossfold(indiv, path, fold, resultsPath, testing_name):
    if indiv['output_dim'] == 50:
        embedding_mtx = fold['embedding_matrix'][0]
    elif indiv['output_dim'] == 100:
        embedding_mtx = fold['embedding_matrix'][1]
    elif indiv['output_dim'] == 200:
        embedding_mtx = fold['embedding_matrix'][2]
    elif indiv['output_dim'] == 300:
        embedding_mtx = fold['embedding_matrix'][3]

    model = cnn.cnn_model(fold['vocab_size'], fold['maxlen'], embedding_mtx,
                          indiv, path)
    
    #early stopping
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_f1_m', mode='max', verbose=True, patience=10)]

    #save the best model
    callbacks += [tf.keras.callbacks.ModelCheckpoint(resultsPath + testing_name + ".h5", monitor='val_f1_m', mode='max', verbose=False, 
                                  save_best_only=True)]

    class_weight = {0: 0.25,
                    1: 0.75}
    model.fit(fold['X_train'], fold['y_train'], epochs=indiv['epochs'], verbose=False, 
              validation_data=(fold['X_val'], fold['y_val']), use_multiprocessing=False,
              batch_size=indiv['batch_size'], callbacks=callbacks, class_weight=class_weight)
    
    dependencies = {
    'f1_m': util.f1_m
    }

    # load the saved model
    for x in range(0, 4):  # try 4 times
        try:
            # msg.send()
            saved_model = tf.keras.models.load_model(resultsPath + testing_name + ".h5", custom_objects=dependencies)
            str_error = None
        except Exception as e:
            print('An error occurs when loading saved model.')
            str_error = e
            pass

        if str_error:
            sleep(2)  # wait for 2 seconds before trying to fetch the data again
        else:
            break
    

    y_pred = saved_model.predict_classes(fold['X_val'])

    os.remove(resultsPath + testing_name + ".h5")

    # CNN metrics
    accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary = util.get_testing_metric(fold['y_val'],
                                                                                                    y_pred)
    return f1ScoreBinary


# Genetic Algorithm

In [6]:
import random
from operator import attrgetter
from deap import base
from deap import creator
from deap import tools
import time
import datetime
import math
from scipy.spatial import distance
import itertools


class GeneticAlgorithm:
    __slots__ = (
        "toolbox", "toolboxes", "cross_rate", "mut_rate", "n_pop", "n_gen", "resultsPath", "testing_name", "cfold",
        "globalparameters", "layerparameters", "defaultVal", "path_ind", "max_conv_idx", "max_maxpooling_idx",
        "max_dense_idx", "max_dropout_idx")

    def __init__(self, toolbox, toolboxes, cross_rate, mut_rate, n_pop, n_gen, resultsPath, testing_name,
                 cfold, globalparameters, layerparameters, defaultVal, path_ind, max_conv_idx, max_maxpooling_idx,
                 max_dense_idx, max_dropout_idx):
        self.toolbox = toolbox
        self.toolboxes = toolboxes
        self.cross_rate = cross_rate
        self.mut_rate = mut_rate
        self.n_pop = n_pop
        self.n_gen = n_gen
        self.resultsPath = resultsPath
        self.testing_name = testing_name
        self.cfold = cfold
        self.globalparameters = globalparameters
        self.layerparameters = layerparameters
        self.defaultVal = defaultVal
        self.path_ind = path_ind
        self.max_conv_idx = max_conv_idx
        self.max_maxpooling_idx = max_maxpooling_idx
        self.max_dense_idx = max_dense_idx
        self.max_dropout_idx = max_dropout_idx

    def fitnessCalc(self, individual):
        i = 0
        if len(individual.fitness.values) == 0:
            if (0 in individual or '' in individual or 'False' in individual or None in individual):
                for param in self.defaultVal:
                    if individual[i] == 0 or individual[i] == '' or individual[i] == 'False' or individual[i] == None:
                        individual[i] = self.defaultVal[param]
                    i += 1

            fc = FitnessCalculation(individual, self.cfold, self.defaultVal, self.resultsPath, self.testing_name)
        else:
            fc = individual.fitness.values[0]
        print('{} {}'.format(datetime.datetime.now(), fc))
        return fc,

    def write_result(self):
        # Create Testing Results
        f = open(self.resultsPath + self.testing_name + ".csv", "a+")
        text = "i,min,max,mean,std,avgdistance,time,CR,MR"
        for param in self.defaultVal:
            text += ",{0}".format(param)
        text += "\n"       
        f.write(text)
        f.close()

        # Create Last Population file
        f = open(self.resultsPath + self.testing_name + "lastpop.csv", 'a+')
        text = "i,f1score"
        for param in self.defaultVal:
            text += ",{0}".format(param)
        text += "\n"
        f.write(text)
        f.close()

    def std_calc(self, fits, length):
        mean = sum(fits) / length
        sum2 = sum(x * x for x in fits)
        std = abs(sum2 / length - mean ** 2) ** 0.5

        return mean, std
    
    def distance_calc(self, pop):
        distances = []
        for subset in itertools.combinations(pop, 2):
            distances.append(distance.hamming(subset[0][0:subset[0].index('embedding_layer')],
                                              subset[1][0:subset[1].index('embedding_layer')]))

        avgDistance = sum(distances) / len(distances)
        
        return avgDistance

    def invalid_fitness_calc(self, pop):
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in pop if not ind.fitness.valid]
        fitnesses = map(self.toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

    def mutHyperparam(self, individual, indpb):
        toolboxesSize = len(self.toolboxes)
        fsm = FSM()
        mutatefsm = mutateFSM()
        addfsm = addFSM()

        # Mutation for the Hyperparameter Chromosomes
        for i in range(toolboxesSize):
            if random.random() < indpb:
                if len(self.toolboxes[i].args) == 1:
                    individual[i] = self.toolboxes[i].func(self.toolboxes[i].args[0])
                else:
                    individual[i] = self.toolboxes[i].func(self.toolboxes[i].args[0], self.toolboxes[i].args[1])

        # Mutation for the Architecture Chromosomes
        archChrom = individual[individual.index('convolutional_layer'):individual.index('output_layer')]
        size = len(archChrom)

        for i in range(1, size):
            if random.random() < indpb:
                if (i>=size):
                  break
                
                if (archChrom[i] == 'global_maxpooling_layer'):
                  continue

                selectMutType = random.randint(0, 2)
                # Remove the layer
                if selectMutType == 0:
                  for key in fsm:
                      if fsm[key]['layer'] in archChrom[i - 1:i]:
                          for j in fsm[key]['next_path']:
                              if i != size - 1 and i + 1 < size and fsm[j]['layer'] == archChrom[i + 1]:
                                  archChrom.remove(archChrom[i])
#                                   print('individual before remove', individual)
                                  individual[
                                  individual.index('convolutional_layer'):individual.index('output_layer')] = archChrom
#                                   print('individual after remove', individual)
                                  size -= 1          
                                  break
                          else:
                            continue
                          break                              

                # Change the layer
                elif selectMutType == 1:
                    if i == size - 1:
                        if (archChrom[i] == 'dropout_layer') or (archChrom[i] == 'dense_layer' and 'dropout_layer' not in archChrom[i - 1:i]):
                            if (mutatefsm[archChrom[i]]['change'] == 'convolutional_layer' and individual.count(
                                    'convolutional_layer') < self.max_conv_idx) or (
                                    mutatefsm[archChrom[i]]['change'] == 'dense_layer' and individual.count('dense_layer')
                                    < self.max_dense_idx) or (
                                    mutatefsm[archChrom[i]]['change'] == 'maxpooling_layer' and individual.count(
                                'maxpooling_layer') < self.max_maxpooling_idx) or (
                                    mutatefsm[archChrom[i]]['change'] == 'dropout_layer' and individual.count('dropout_layer')
                                    < self.max_dropout_idx):
                                archChrom[i] = mutatefsm[archChrom[i]]['change']
#                                 print('individual before change', individual)
                                individual[
                                individual.index('convolutional_layer'):individual.index('output_layer')] = archChrom
#                                 print('individual after change', individual)

                    else:
                        if all(item in mutatefsm[archChrom[i]]['before'] for item in archChrom[i - 1:i]) and archChrom[i + 1] in mutatefsm[archChrom[i]]['after']:
                            if (mutatefsm[archChrom[i]]['change'] == 'convolutional_layer' and individual.count(
                                    'convolutional_layer') < self.max_conv_idx) or (
                                    mutatefsm[archChrom[i]]['change'] == 'dense_layer' and individual.count('dense_layer')
                                    < self.max_dense_idx) or (
                                    mutatefsm[archChrom[i]]['change'] == 'maxpooling_layer' and individual.count(
                                'maxpooling_layer') < self.max_maxpooling_idx) or (
                                    mutatefsm[archChrom[i]]['change'] == 'dropout_layer' and individual.count('dropout_layer')
                                    < self.max_dropout_idx):
                                
                                archChrom[i] = mutatefsm[archChrom[i]]['change']
#                                 print('individual before change', individual)
                                individual[
                                individual.index('convolutional_layer'):individual.index('output_layer')] = archChrom
#                                 print('individual after change', individual)
                # Add a layer
                elif selectMutType == 2:
                    for key in addfsm:
                        if key in archChrom[i] and all(item in addfsm[archChrom[i]]['before'] for item in archChrom[i - 1:i]):
                            if ('convolutional_layer' in addfsm[archChrom[i]]['add'] and individual.count(
                                    'convolutional_layer') < self.max_conv_idx) or (
                                    'dense_layer' in addfsm[archChrom[i]]['add'] and individual.count('dense_layer') < self.max_dense_idx) or (
                                    'maxpooling_layer' in addfsm[archChrom[i]]['add'] and individual.count(
                                'maxpooling_layer') < self.max_maxpooling_idx) or (
                                    'dropout_layer' in addfsm[archChrom[i]]['add'] and individual.count(
                                'dropout_layer') < self.max_dropout_idx):
                                archChrom.insert(i, random.choice(addfsm[archChrom[i]]['add']))
#                                 print('individual before add', individual)
                                individual[
                                individual.index('convolutional_layer'):individual.index('output_layer')] = archChrom
#                                 print('individual after add', individual)
                        
        return individual,

    def cxTwoPoint(self, ind1, ind2, pop, offspring):
        # Crossover for hyperparameter chromosomes
        size = ind1.index('embedding_layer')
        selectCxType = random.randint(0, 2)
        # One point crossover
        if selectCxType == 0:
#             print('ind1 before one-point crossover:', ind1)
#             print('ind2 before one-point crossover:', ind2)
            cxpoint = random.randint(1, size - 1)
            ind1[cxpoint:], ind2[cxpoint:] = ind2[cxpoint:], ind1[cxpoint:]
#             print('ind1 after one-point crossover:', ind1)
#             print('ind2 after one-point crossover:', ind2)
        # Two-point crossover
        elif selectCxType == 1:
#             print('ind1 before two-point crossover:', ind1)
#             print('ind2 before two-point crossover:', ind2)
            cxpoint1 = random.randint(1, size - 1)
            cxpoint2 = random.randint(1, size - 1)
            if cxpoint2 >= cxpoint1:
                cxpoint2 += 1
            else:  # Swap the two cx points
                cxpoint1, cxpoint2 = cxpoint2, cxpoint1

            ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] \
                = ind2[cxpoint1:cxpoint2], ind1[cxpoint1:cxpoint2]
#             print('ind1 after two-point crossover:', ind1)
#             print('ind2 after two-point crossover:', ind2)
        # Uniform crossover
        elif selectCxType == 2:
#             print('ind1 before uniform crossover:', ind1)
#             print('ind2 before uniform crossover:', ind2)
            for i in range(size):
                if random.random() < self.cross_rate:
                    ind1[i], ind2[i] = ind2[i], ind1[i]
#             print('ind1 after uniform crossover:', ind1)
#             print('ind2 after uniform crossover:', ind2)

        # Crossover for architecture chromosomes
        # One-cut point crossover from the Global MaxPooling layer
        cxpoint1 = ind1.index('global_maxpooling_layer')
        cxpoint2 = ind2.index('global_maxpooling_layer')
        ind1[cxpoint1:], ind2[cxpoint2:] = ind2[cxpoint2:], ind1[cxpoint1:]

        max_drop_layer = max(ind1.count('dropout_layer'), ind2.count('dropout_layer'))
        if max_drop_layer > self.max_dropout_idx:
            idx = self.max_dropout_idx
            self.max_dropout_idx = max_drop_layer

            while idx < self.max_dropout_idx:
                idx += 1
                addDropoutLayer(idx, self.toolbox, self.toolboxes, self.defaultVal, self.layerparameters)

                for ind in pop + offspring:
                    ind.insert(size, random.uniform(0, 1))
                size += 1

                self.write_result()

        return ind1, ind2

    def runGA(self, lastPop=[], lastFitnesses=[]):
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMax)

        self.toolbox.register("individual", tools.initCycle, creator.Individual,
                              self.toolboxes, n=1)
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        self.toolbox.register("evaluate", self.fitnessCalc)
        self.toolbox.register("mate", self.cxTwoPoint)
        self.toolbox.register("mutate", self.mutHyperparam, indpb=self.mut_rate)
        self.toolbox.register("select", tools.selBest)

        pop = self.toolbox.population(n=self.n_pop)

        idx = 0
        for ind in pop:
            if lastPop:
                ind[:] = lastPop[idx]
            ind.extend(self.path_ind[idx])
            idx += 1
        
        if lastFitnesses:
            # Fitnesses from previous population
            fitnesses = lastFitnesses
        else:
            # Evaluate the entire population
            fitnesses = list(map(self.toolbox.evaluate, pop))

        for ind, fit in zip(pop, fitnesses):
            ind.fitness.values = fit

        self.write_result()
        
        g = 0
        increment = 0.01
        avgDistance = 1
        threshold = 0.5
        minRate = 0.01
        maxRate = 1.0
        
        while g < self.n_gen:
            then = time.time()
            g = g + 1
            print('{} {}'.format(datetime.datetime.now(), "-- Generation %i --" % g))
            
            if g > 1:
                # Adaptive crossover and mutation rates
                print('Crossover rate before adaptive scheme=', self.cross_rate)
                print('Mutation rate before adaptive scheme=', self.mut_rate)   
                
                if avgDistance > threshold:
                    self.cross_rate += increment
                    self.mut_rate -= increment                    
                else:
                    self.cross_rate -= increment
                    self.mut_rate += increment                    
                    
                if self.mut_rate > maxRate:
                    self.mut_rate = maxRate
                elif self.mut_rate < minRate:
                    self.mut_rate = minRate

                if self.cross_rate > maxRate:
                    self.cross_rate = maxRate
                elif self.cross_rate < minRate:
                    self.cross_rate = minRate
                
                print('Crossover rate after adaptive scheme=', self.cross_rate)
                print('Mutation rate after adaptive scheme=', self.mut_rate)
            
            # Select the next generation individuals
            offspring = self.toolbox.select(pop, len(pop))
            # Clone the selected individuals
            offspring = list(map(self.toolbox.clone, offspring))

            # Apply crossover and mutation on the offspring
            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                if random.random() < self.cross_rate:
                    self.toolbox.mate(child1, child2, pop, offspring)
                    del child1.fitness.values
                    del child2.fitness.values

            for mutant in offspring:
                if random.random() < self.mut_rate:
                    self.toolbox.mutate(mutant)
                    del mutant.fitness.values

            # Evaluate the individuals with an invalid fitness
            self.invalid_fitness_calc(offspring)

            pop[:] = self.toolbox.select(pop + offspring, self.n_pop)

            # Gather all the fitnesses in one list and print the stats
            fits = [ind.fitness.values[0] for ind in pop]

            length = len(pop)
            mean, std = self.std_calc(fits, length)
            avgDistance = self.distance_calc(pop)
            best = max(pop, key=attrgetter("fitness"))
            print('{} {}'.format(datetime.datetime.now(), "  Min %s" % min(fits)))
            print('{} {}'.format(datetime.datetime.now(), "  Max %s" % max(fits)))
            print('{} {}'.format(datetime.datetime.now(), "  Avg %s" % mean))
            print('{} {}'.format(datetime.datetime.now(), "  Std %s" % std))
            print('{} {}'.format(datetime.datetime.now(), "  AvgDistance %s" % avgDistance))
            print('{} {}'.format(datetime.datetime.now(), best))

            now = time.time()
            diff = now - then

            # save testing data
            f = open(self.resultsPath + self.testing_name + ".csv", 'a')
            text = "{0},{1},{2},{3},{4},{5},{6},{7},{8}".format(g,min(fits), max(fits), mean, std, avgDistance, diff, self.cross_rate, self.mut_rate)
            for param in best:
                text += ",{0}".format(param)
            text += "\n"
            f.write(text)
            f.close()

            # save last population data
            f = open(self.resultsPath + self.testing_name + "lastpop.csv", 'a')
            for ind in pop:
                text = "{0},{1}".format(g,ind.fitness.values[0])
                for param in ind:
                    text += ",{0}".format(param)
                text += "\n"                 
                f.write(text)

            f.close()            

# Project path

In [7]:
training_path = 'TrainingData.csv'
trial_path = 'TrialData.csv'
population_path = 'NewPop.csv'
root_path = '/lab/dbms/fatyanosa'
datasetPath = root_path + '/Dataset/'
resultsPath = root_path + '/Server1/Results/'
archPath = root_path + '/Server1/Architecture/'
testing_name = "GA-CNN with Adaptive CRMR"
glovePath = [root_path + '/Glove/glove.6B.50d.txt',
                 root_path + '/Glove/glove.6B.100d.txt',
                 root_path + '/Glove/glove.6B.200d.txt',
                 root_path + '/Glove/glove.6B.300d.txt']

# Parameters

In [8]:
# crossover rate is the probability with which two individuals
cross_rate = 0.8

# mutation rate is the probability for mutating an individual
mut_rate = 0.2

# number of population
n_pop = 30

# number of generation
n_gen = 100

# Main Program

In [9]:
import random
from sklearn.model_selection import StratifiedKFold
from deap import base
import warnings; warnings.simplefilter('ignore')

if __name__ == '__main__':
    globalparameters = []
    globalparameters.append(("epochs", random.randint, 1, 100))
    globalparameters.append(("batch_size", random.randint, 32, 256))

    globalparameters.append(("optimizer", random.choice, ['sgd', 'rmsprop', 'adagrad', 'adadelta', 'adam',
                                                          'adamax', 'nadam']))
    globalparameters.append(("learning_rate", random.uniform, 1e-4, 1e-2))
    globalparameters.append(("momentum", random.uniform, 0, 1))
    globalparameters.append(("output_init_mode", random.choice,
                             ['zeros',
                              'ones',
                              'uniform',
                              'normal',
                              'glorot_normal',
                              'glorot_uniform',
                              'he_normal',
                              'he_uniform',
                              'lecun_normal',
                              'lecun_uniform']))
    globalparameters.append(("output_dim", random.choice, [50, 100, 200, 300]))

    layerparameters = {}
    layerparameters["num_filters"] = [random.randint, 32, 512]
    layerparameters["kernel_size"] = [random.randint, 1, 5]
    layerparameters["conv_activation_func"] = [random.choice,
                                               ['relu', 'softmax', 'elu', 'selu',
                                                'softplus', 'softsign', 'tanh',
                                                'sigmoid', 'hard_sigmoid', 'linear']]
    layerparameters["conv_init_mode"] = [random.choice,
                                         ['zeros',
                                          'ones',
                                          'uniform',
                                          'normal',
                                          'glorot_normal',
                                          'glorot_uniform',
                                          'he_normal',
                                          'he_uniform',
                                          'lecun_normal',
                                          'lecun_uniform']]
    layerparameters["conv_weight_constraint"] = [random.randint, 1, 5]
    layerparameters["neurons"] = [random.randint, 1, 30]
    layerparameters["dense_activation_func"] = [random.choice,
                                                ['relu', 'softmax', 'elu', 'selu',
                                                 'softplus', 'softsign', 'tanh',
                                                 'sigmoid', 'hard_sigmoid', 'linear']]
    layerparameters["dense_init_mode"] = [random.choice,
                                          ['zeros',
                                           'ones',
                                           'uniform',
                                           'normal',
                                           'glorot_normal',
                                           'glorot_uniform',
                                           'he_normal',
                                           'he_uniform',
                                           'lecun_normal',
                                           'lecun_uniform']]
    layerparameters["dense_weight_constraint"] = [random.randint, 1, 5]
    layerparameters["pool_size"] = [random.randint, 2, 6]
    layerparameters["dropout_rate"] = [random.uniform, 0, 1]

    defaultVal = collections.OrderedDict([
        ("epochs", 10),
        ("batch_size", 32),
        ("optimizer", "adam"),
        ("learning_rate", 1e-4),
        ("momentum", 0.9),
        ("output_init_mode", "glorot_uniform"),
        ("output_dim", 100)]
    )
    
    # object class
    util = utility()
    toolbox = base.Toolbox()
    toolboxes = []

    # Attribute generator
    for hyper in globalparameters:
        if len(hyper) == 3:
            toolbox.register(hyper[0], hyper[1], hyper[2])
        else:
            toolbox.register(hyper[0], hyper[1], hyper[2], hyper[3])

    toolboxes.append(toolbox.epochs)
    toolboxes.append(toolbox.batch_size)
    toolboxes.append(toolbox.optimizer)
    toolboxes.append(toolbox.learning_rate)
    toolboxes.append(toolbox.momentum)
    toolboxes.append(toolbox.output_init_mode)
    toolboxes.append(toolbox.output_dim)

    path_ind, max_conv_idx, max_maxpooling_idx, max_dense_idx, max_dropout_idx = generateFSM(n_pop, layerparameters,
                                                                                             toolbox, toolboxes,
                                                                                             defaultVal)

#     # Read population data
#     dfPopulation = util.read_CSV(resultsPath + population_path)

#     path_ind, fitnesses, max_conv_idx, max_maxpooling_idx, max_dense_idx, max_dropout_idx = openFSM(dfPopulation, layerparameters,
#                                                                                          toolbox, toolboxes, defaultVal)
#     dfPopulation = dfPopulation.drop(columns=[col for col in dfPopulation if col not in defaultVal])

#     population = dfPopulation.loc[:, ~dfPopulation.columns.str.match('Unnamed')].values.tolist()

    # Read data
    dfTraining = util.read_CSV(datasetPath + training_path)
    
    # Read trial data
    dfTrial = util.read_CSV(datasetPath + trial_path)

    textsTraining, labelsTraining = util.get_text_label(dfTraining)
    textsTrial, labelsTrial = util.get_text_label(dfTrial)
    cfold = {}

    X_train, X_val, y_train, y_val, vocab_size, maxlen, embedding_matrix = util.get_training_trial_data(
        textsTraining, labelsTraining, textsTrial, labelsTrial, glovePath)
    cfold= {'X_train': X_train, 'X_val': X_val, 'y_train': y_train, 'y_val': y_val, 'vocab_size': vocab_size,
                  'maxlen': maxlen, 'embedding_matrix': embedding_matrix}
                  
    ga = GeneticAlgorithm(toolbox, toolboxes, cross_rate, mut_rate, n_pop, n_gen, resultsPath, testing_name,
                          cfold, globalparameters, layerparameters, defaultVal, path_ind, max_conv_idx, max_maxpooling_idx, max_dense_idx, max_dropout_idx)
    ga.runGA()
#     ga.runGA(population, fitnesses)


Epoch 00011: early stopping
2020-02-28 19:36:48.336801 0.6666666666666666
2020-02-28 19:36:53.118247 0.6620947630922693
Epoch 00011: early stopping
2020-02-28 19:37:33.259532 0.6666666666666666
Epoch 00016: early stopping
2020-02-28 19:37:57.665219 0.625776397515528
Epoch 00012: early stopping
2020-02-28 19:38:21.476554 0.6996838777660696
Epoch 00011: early stopping
2020-02-28 19:38:42.661715 0.6666666666666666
Epoch 00011: early stopping
2020-02-28 19:39:12.658016 0.6666666666666666
Epoch 00011: early stopping
2020-02-28 19:39:36.735385 0.6666666666666666
2020-02-28 19:39:44.689806 0.6666666666666666
Epoch 00011: early stopping
2020-02-28 19:41:00.341672 0.6666666666666666
Epoch 00011: early stopping
2020-02-28 19:41:22.843318 0.6666666666666666
Epoch 00011: early stopping
2020-02-28 19:42:46.920732 0.6666666666666666
Epoch 00011: early stopping
2020-02-28 19:43:00.547026 0.0
Epoch 00011: early stopping
2020-02-28 19:43:14.924383 0.0
2020-02-28 19:43:59.847610 0.5855855855855856
Epoch

Epoch 00020: early stopping
2020-02-28 20:12:39.190775 0.6666666666666666
2020-02-28 20:13:08.782451 0.6876340243030736
Epoch 00011: early stopping
2020-02-28 20:13:32.879046 0.6666666666666666
Epoch 00011: early stopping
2020-02-28 20:13:43.585269 0.7031963470319634
2020-02-28 20:13:47.546551 0.7003891050583657
Epoch 00020: early stopping
2020-02-28 20:14:25.551306 0.6729435084241823
2020-02-28 20:14:29.109729 0.2368421052631579
Epoch 00011: early stopping
2020-02-28 20:14:51.319403 0.6666666666666666
Epoch 00011: early stopping
2020-02-28 20:15:13.353345 0.6623931623931624
Epoch 00011: early stopping
2020-02-28 20:15:22.299363 0.0028248587570621473
Epoch 00011: early stopping
2020-02-28 20:15:48.533424 0.6786659608258337
2020-02-28 20:15:58.217663 0.540765391014975
Epoch 00011: early stopping
2020-02-28 20:16:30.216787 0.6666666666666666
Epoch 00027: early stopping
2020-02-28 20:17:16.368689 0.6595264937993235
Epoch 00017: early stopping
2020-02-28 20:17:35.974190 0.14285714285714285

Epoch 00015: early stopping
2020-02-28 20:37:48.219664 0.6951871657754011
Epoch 00013: early stopping
2020-02-28 20:38:14.709535 0.7004805125467165
2020-02-28 20:38:18.500383 0.4153225806451613
Epoch 00011: early stopping
2020-02-28 20:38:41.065811 0.6858846918489065
Epoch 00011: early stopping
2020-02-28 20:38:52.201453 0.6666666666666666
2020-02-28 20:38:56.140293 0.07660738714090286
2020-02-28 20:38:59.773307 0.4950316169828365
2020-02-28 20:38:59.811092   Min 0.695945945945946
2020-02-28 20:38:59.811670   Max 0.7194492254733219
2020-02-28 20:38:59.811693   Avg 0.7076528465383448
2020-02-28 20:38:59.812068   Std 0.007519821384022586
2020-02-28 20:38:59.812091   AvgDistance 0.7638100067613246
2020-02-28 20:38:59.812108 [20, 141, 'nadam', 0.0031500776388229627, 0.26437922170291406, 'glorot_uniform', 100, 341, 2, 'relu', 'glorot_uniform', 3, 111, 4, 'softmax', 'ones', 5, 92, 5, 'softmax', 'glorot_uniform', 2, 256, 4, 'softsign', 'he_normal', 4, 382, 3, 'hard_sigmoid', 'glorot_uniform',

Epoch 00013: early stopping
2020-02-28 20:53:53.933178 0.6911157024793387
Epoch 00011: early stopping
2020-02-28 20:54:08.947801 0.6666666666666666
Epoch 00013: early stopping
2020-02-28 20:54:20.979916 0.6782178217821782
Epoch 00015: early stopping
2020-02-28 20:54:35.745388 0.6666666666666666
Epoch 00011: early stopping
2020-02-28 20:54:51.500705 0.7318208260616638
Epoch 00014: early stopping
2020-02-28 20:55:16.872510 0.7163515016685206
Epoch 00011: early stopping
2020-02-28 20:55:30.613424 0.7154271665642287
Epoch 00016: early stopping
2020-02-28 20:55:54.625047 0.6845084920226455
2020-02-28 20:56:17.474773 0.6692209450830141
Epoch 00013: early stopping
2020-02-28 20:56:33.525618 0.5674008810572687
Epoch 00011: early stopping
2020-02-28 20:56:48.630521 0.7374860956618464
Epoch 00015: early stopping
2020-02-28 20:57:18.466552 0.7015590200445434
Epoch 00015: early stopping
2020-02-28 20:57:29.457496 0.7107969151670951
Epoch 00014: early stopping
2020-02-28 20:57:46.029650 0.661495063

Epoch 00013: early stopping
2020-02-28 21:14:01.620337 0.713641488162345
Epoch 00016: early stopping
2020-02-28 21:14:19.822279 0.6824034334763948
Epoch 00013: early stopping
2020-02-28 21:14:43.008476 0.6666666666666666
Epoch 00016: early stopping
2020-02-28 21:14:54.478763 0.6037441497659906
Epoch 00011: early stopping
2020-02-28 21:15:13.183292 0.6666666666666666
Epoch 00014: early stopping
2020-02-28 21:15:27.402859 0.6888135593220339
Epoch 00016: early stopping
2020-02-28 21:15:42.993979 0.6666666666666666
Epoch 00012: early stopping
2020-02-28 21:15:55.736933 0.6743961352657004
Epoch 00012: early stopping
2020-02-28 21:16:07.809738 0.698515171078115
Epoch 00013: early stopping
2020-02-28 21:16:21.244720 0.6645202833226014
Epoch 00012: early stopping
2020-02-28 21:16:36.149657 0.7334963325183375
Epoch 00012: early stopping
2020-02-28 21:16:50.912798 0.7209595959595959
2020-02-28 21:16:50.951549   Min 0.7157894736842104
2020-02-28 21:16:50.951957   Max 0.7374860956618464
2020-02-28

...

Epoch 00011: early stopping
2020-02-29 12:10:34.202091 0.6867887184562097
Epoch 00015: early stopping
2020-02-29 12:11:01.519055 0.6355555555555557
Epoch 00012: early stopping
2020-02-29 12:11:26.367417 0.7235641438539989
Epoch 00012: early stopping
2020-02-29 12:11:52.389524 0.7335211267605634
Epoch 00011: early stopping
2020-02-29 12:12:17.631749 0.6918809884014119
Epoch 00011: early stopping
2020-02-29 12:12:40.859756 0.6928972917731221
Epoch 00011: early stopping
2020-02-29 12:13:07.191485 0.6647591797806389
Epoch 00016: early stopping
2020-02-29 12:13:35.436726 0.7067254685777288
Epoch 00015: early stopping
2020-02-29 12:14:02.916024 0.6911142454160789
Epoch 00011: early stopping
2020-02-29 12:14:26.081522 0.6682600382409177
Epoch 00011: early stopping
2020-02-29 12:14:49.115127 0.6969535008017101
Epoch 00013: early stopping
2020-02-29 12:15:17.188957 0.7126436781609196
2020-02-29 12:15:17.229061   Min 0.7522236340533673
2020-02-29 12:15:17.229130   Max 0.7531992687385741
2020-02-

Epoch 00015: early stopping
2020-02-29 12:39:34.938291 0.7175398633257403
Epoch 00014: early stopping
2020-02-29 12:40:01.502339 0.6973865199449794
Epoch 00023: early stopping
2020-02-29 12:40:48.558118 0.6888150609080842
Epoch 00013: early stopping
2020-02-29 12:41:23.071135 0.7405281285878299
Epoch 00018: early stopping
2020-02-29 12:41:52.394799 0.7074235807860262
Epoch 00011: early stopping
2020-02-29 12:42:15.276064 0.7231947483588622
Epoch 00012: early stopping
2020-02-29 12:42:39.349547 0.6666666666666666
Epoch 00024: early stopping
2020-02-29 12:43:13.905949 0.6692913385826772
Epoch 00012: early stopping
2020-02-29 12:43:39.395141 0.7492795389048993
Epoch 00013: early stopping
2020-02-29 12:44:06.447127 0.7392619479733817
Epoch 00013: early stopping
2020-02-29 12:44:33.121787 0.742890307603018
Epoch 00019: early stopping
2020-02-29 12:45:04.520001 0.6772546795235393
Epoch 00013: early stopping
2020-02-29 12:45:28.328916 0.7231430325352978
Epoch 00017: early stopping
2020-02-29 

2020-02-29 13:09:49.816003 0.6666666666666666
Epoch 00013: early stopping
2020-02-29 13:10:16.207439 0.7481751824817517
Epoch 00011: early stopping
2020-02-29 13:10:45.233315 0.6881720430107526
Epoch 00016: early stopping
2020-02-29 13:11:13.530024 0.6931659693165968
Epoch 00014: early stopping
2020-02-29 13:11:41.005950 0.6905503634475596
Epoch 00014: early stopping
2020-02-29 13:12:08.832453 0.6914498141263941
Epoch 00012: early stopping
2020-02-29 13:12:35.029986 0.736842105263158
Epoch 00014: early stopping
2020-02-29 13:13:03.787510 0.7058217413704276
Epoch 00012: early stopping
2020-02-29 13:13:30.696334 0.6705035971223021
Epoch 00016: early stopping
2020-02-29 13:14:01.876648 0.6593856655290102
Epoch 00012: early stopping
2020-02-29 13:14:52.331050 0.5661375661375662
Epoch 00012: early stopping
2020-02-29 13:15:20.117700 0.7493333333333333
Epoch 00011: early stopping
2020-02-29 13:15:43.945794 0.691073219658977
Epoch 00013: early stopping
2020-02-29 13:17:05.067629 0.725
Epoch 0

Epoch 00014: early stopping
2020-02-29 13:41:28.343163 0.7004175365344468
Epoch 00018: early stopping
2020-02-29 13:41:58.123479 0.6890547263681591
Epoch 00012: early stopping
2020-02-29 13:42:30.247894 0.7293956043956042
Epoch 00013: early stopping
2020-02-29 13:43:03.974862 0.7408737283064034
Epoch 00011: early stopping
2020-02-29 13:43:34.095448 0.6666666666666666
Epoch 00021: early stopping
2020-02-29 13:44:17.211174 0.702247191011236
Epoch 00012: early stopping
2020-02-29 13:44:40.998070 0.7189542483660131
Epoch 00013: early stopping
2020-02-29 13:45:07.144342 0.722755505364201
Epoch 00013: early stopping
2020-02-29 13:45:30.992524 0.7309192200557103
Epoch 00011: early stopping
2020-02-29 13:45:53.547091 0.7172859450726979
Epoch 00011: early stopping
2020-02-29 13:46:17.924457 0.6866028708133971
Epoch 00014: early stopping
2020-02-29 13:47:02.748724 0.6777668952007836
Epoch 00013: early stopping
2020-02-29 13:47:27.239610 0.7317608337904553
Epoch 00012: early stopping
2020-02-29 1

Epoch 00013: early stopping
2020-02-29 14:14:37.694608 0.7039904705181657
Epoch 00014: early stopping
2020-02-29 14:15:08.794248 0.6494762288477035
Epoch 00012: early stopping
2020-02-29 14:15:36.203916 0.6763580719204285
Epoch 00012: early stopping
2020-02-29 14:16:02.349854 0.6799999999999999
Epoch 00012: early stopping
2020-02-29 14:16:27.948166 0.7228525121555915
2020-02-29 14:16:27.984136   Min 0.7522236340533673
2020-02-29 14:16:27.984190   Max 0.7531992687385741
2020-02-29 14:16:27.984218   Avg 0.7527219222429109
2020-02-29 14:16:27.984232   Std 0.00031580392370987966
2020-02-29 14:16:27.984248   AvgDistance 0.30851926977687627
2020-02-29 14:16:27.984270 [20, 141, 'nadam', 0.0020138629223913755, 0.8968932676829043, 'glorot_uniform', 100, 153, 2, 'elu', 'he_uniform', 3, 195, 4, 'softmax', 'ones', 5, 335, 3, 'softmax', 'glorot_uniform', 2, 256, 4, 'softsign', 'glorot_normal', 4, 382, 3, 'hard_sigmoid', 'glorot_uniform', 1, 298, 1, 'linear', 'uniform', 4, 276, 1, 'sigmoid', 'ones',